The data utilized for this analysis was scraped from the FBRef website (https://fbref.com/en/), a reputable source for football statistics and history. The focus of this project was on gathering and analyzing individual team match data from the Premier League, spanning from the 2016-17 season to the current season, 2023-24. In addition to the points table information, we also retrieved crucial shooting statistics for each team in every match from the shooting table. This comprehensive dataset allows for a detailed exploration of team performance metrics and shooting proficiency within the Premier League during the specified timeframe.

### Scraping page with requests

In [96]:
import requests

In [97]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [98]:
# Download the page
data = requests.get(standings_url)

In [99]:
print(data) # The response should be '200'. If it gives '429', then try again later as it is rate limiting.

<Response [200]>


### Parsing HTML links with BeautifulSoup

In [100]:
from bs4 import BeautifulSoup

In [101]:
import time
#Giving the browser time to complete the action and then come up with new sourcecode before reading it using beautifulsoup

soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [102]:
team_urls = [f"https://fbref.com{l}" for l in links]

### Extracting match stats using pandas and requests

In [103]:
data = requests.get(team_urls[0])

In [104]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

### Getting match shooting stats with requests and pandas

In [105]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [106]:
data = requests.get(f"https://fbref.com{links[0]}")

In [107]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [108]:
shooting.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2023-08-06  16:00  Community Shield  FA Community Shield  Sun   
1          2023-08-11  20:00    Premier League          Matchweek 1  Fri   
2          2023-08-16  22:00         Super Cup       UEFA Super Cup  Wed   
3          2023-08-19  20:00    Premier League          Matchweek 2  Sat   
4          2023-08-27  14:00    Premier League          Matchweek 3  Sun   

                                                ... Standard                \
     Venue Result     GF     GA       Opponent  ...     Dist   FK PK PKatt   
0  Neutral      D  1 (1)  1 (4)        Arsenal  ...      NaN  NaN  0     0   
1     Away      W      3      0        Burnley  ...     13.9  0.0  0     0   
2     Home      D  1 (5)  1 (4)     es Sevilla  ...      NaN  NaN  0     0   
3     Home      W      1      0  Newcastle Utd  ...     17.9  0.0  0     0   
4     Away      W      2      1  Sheffield Utd  ...     17.3  2.0  0     1   

  Expected                           Unnamed: 25_level_0  
        xG npxG npxG/Sh G-xG np:G-xG        Match Report  
0      NaN  NaN     NaN  NaN     NaN        Match Report  
1      1.9  1.9    0.12  1.1     1.1        Match Report  
2      NaN  NaN     NaN  NaN     NaN        Match Report  
3      1.0  1.0    0.07  0.0     0.0        Match Report  
4      3.5  2.8    0.10 -1.5    -0.8        Match Report  

[5 rows x 26 columns]

### Cleaning and merging scraped data with pandas

In [109]:
shooting.columns = shooting.columns.droplevel()

In [110]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [111]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,4-2-3-1,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...,8,4,NaN,NaN,0,0
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,4-2-3-1,Craig Pawson,Match Report,NaN,17,8,13.9,0.0,0,0
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,4-2-3-1,François Letexier,Match Report,NaN,23,7,NaN,NaN,0,0
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,4-2-3-1,Robert Jones,Match Report,NaN,14,4,17.9,0.0,0,0
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,4-2-3-1,Jarred Gillett,Match Report,NaN,29,9,17.3,2.0,0,1


### Scraping data for multiple seasons and teams with a loop

In [112]:
from datetime import datetime
current_year = datetime.now().year
years = list(range(current_year, 2016, -1))
all_matches = []

In [113]:
years #The year'2016' is actually the season '2015-2016'. We use this everywhere which makes working with the data easier. 

[2023, 2022, 2021, 2020, 2019, 2018, 2017]

In [114]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [115]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [116]:
len(all_matches)

140

In [117]:
match_df = pd.concat(all_matches) # Combining the individual dataframes into a single dataframe

### Data Cleaning

In [118]:
match_df.columns = [c.lower() for c in match_df.columns]

In [119]:
match_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0.0,0.0,2023,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0.0,0.0,2023,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0.0,1.0,2023,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1.0,1.0,2023,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0.0,0.0,2023,Manchester City


In [120]:
match_df.shape

(4678, 27)

In [121]:
match_df["team"].value_counts()

Manchester City             234
Chelsea                     234
Brighton and Hove Albion    234
Tottenham Hotspur           234
Arsenal                     234
West Ham United             234
Newcastle United            234
Manchester United           234
Crystal Palace              234
Liverpool                   234
Everton                     234
Southampton                 228
Leicester City              228
Wolverhampton Wanderers     196
Burnley                     195
Bournemouth                 158
Aston Villa                 158
Watford                     152
Fulham                      120
Leeds United                114
Brentford                    82
Sheffield United             82
Norwich City                 76
West Bromwich Albion         76
Huddersfield Town            76
Nottingham Forest            44
Cardiff City                 38
Swansea City                 38
Stoke City                   38
Luton Town                    5
Name: team, dtype: int64

In [122]:
match_df.isnull().sum()

date               0
time               0
comp               0
round              0
day                0
venue              0
result             0
gf                 0
ga                 0
opponent           0
xg                 0
xga                0
poss               0
attendance       884
captain            0
formation          0
referee            0
match report       0
notes           4678
sh                 0
sot                0
dist               3
fk                 0
pk                 0
pkatt              0
season             0
team               0
dtype: int64

In [123]:
match_df.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team'],
      dtype='object')

In [124]:
match_df.rename(columns = {'gf':'goals_scored', 
                'ga':'goals_conceded', 
                'xg':'expected_goals', 
                'xga':'expected_assists', 
                'poss':'possession', 
                'sh':'shots',
                'sot':'shots_on_target',
                'dist':'avg_shot_distance',
                'fk':'freekicks',
                'pk':'penaltykicks',
                'pkatt':'penaltykicks_opponent'},
                inplace=True
               )

In [125]:
match_df.head()

,date,time,comp,round,day,venue,result,goals_scored,goals_conceded,opponent,...,match report,notes,shots,shots_on_target,avg_shot_distance,freekicks,penaltykicks,penaltykicks_opponent,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0.0,0.0,2023,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0.0,0.0,2023,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0.0,1.0,2023,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1.0,1.0,2023,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0.0,0.0,2023,Manchester City


In [126]:
match_df.drop(columns=['notes', 'match report'], inplace=True)

In [127]:
match_df.isnull().sum()

date                       0
time                       0
comp                       0
round                      0
day                        0
venue                      0
result                     0
goals_scored               0
goals_conceded             0
opponent                   0
expected_goals             0
expected_assists           0
possession                 0
attendance               884
captain                    0
formation                  0
referee                    0
shots                      0
shots_on_target            0
avg_shot_distance          3
freekicks                  0
penaltykicks               0
penaltykicks_opponent      0
season                     0
team                       0
dtype: int64

In [128]:
match_df.dtypes

date                      object
time                      object
comp                      object
round                     object
day                       object
venue                     object
result                    object
goals_scored              object
goals_conceded            object
opponent                  object
expected_goals           float64
expected_assists         float64
possession               float64
attendance               float64
captain                   object
formation                 object
referee                   object
shots                    float64
shots_on_target          float64
avg_shot_distance        float64
freekicks                float64
penaltykicks             float64
penaltykicks_opponent    float64
season                     int64
team                      object
dtype: object

In [129]:
match_df[["goals_scored", "goals_conceded"]] = match_df[["goals_scored", "goals_conceded"]].astype(int)

In [130]:
match_df.dtypes

date                      object
time                      object
comp                      object
round                     object
day                       object
venue                     object
result                    object
goals_scored               int32
goals_conceded             int32
opponent                  object
expected_goals           float64
expected_assists         float64
possession               float64
attendance               float64
captain                   object
formation                 object
referee                   object
shots                    float64
shots_on_target          float64
avg_shot_distance        float64
freekicks                float64
penaltykicks             float64
penaltykicks_opponent    float64
season                     int64
team                      object
dtype: object

In [131]:
match_df

,date,time,comp,round,day,venue,result,goals_scored,goals_conceded,opponent,...,formation,referee,shots,shots_on_target,avg_shot_distance,freekicks,penaltykicks,penaltykicks_opponent,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,4-2-3-1,Craig Pawson,17.0,8.0,13.9,0.0,0.0,0.0,2023,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,4-2-3-1,Robert Jones,14.0,4.0,17.9,0.0,0.0,0.0,2023,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,4-2-3-1,Jarred Gillett,29.0,9.0,17.3,2.0,0.0,1.0,2023,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,4-2-3-1,Michael Oliver,6.0,4.0,14.8,0.0,1.0,1.0,2023,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,4-2-3-1,Andy Madley,29.0,13.0,16.4,1.0,0.0,0.0,2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2018-04-15,16:00,Premier League,Matchweek 34,Sun,Away,W,1,0,Manchester Utd,...,4-4-1-1,Paul Tierney,10.0,4.0,18.1,0.0,0.0,0.0,2017,West Bromwich Albion
39,2018-04-21,12:30,Premier League,Matchweek 35,Sat,Home,D,2,2,Liverpool,...,4-4-1-1,Stuart Attwell,13.0,6.0,17.7,0.0,0.0,0.0,2017,West Bromwich Albion
40,2018-04-28,15:00,Premier League,Matchweek 36,Sat,Away,W,1,0,Newcastle Utd,...,4-4-1-1,David Coote,9.0,2.0,20.1,0.0,0.0,0.0,2017,West Bromwich Albion
41,2018-05-05,15:00,Premier League,Matchweek 37,Sat,Home,W,1,0,Tottenham,...,4-4-1-1,Mike Jones,9.0,1.0,10.2,0.0,0.0,0.0,2017,West Bromwich Albion


### Writing the data to a CSV

In [132]:
match_df.to_csv("matches.csv")